# DAT405 Introduction to Data Science and AI 
## Assignment 4: Spam classification using Naïve Bayes 

The exercise takes place in a notebook environment where you can chose to use Jupyter or Google Colabs. We recommend you use Google Colabs as it will facilitate remote group-work and makes the assignment less technical. 
Hints:
You can execute certain linux shell commands by prefixing the command with `!`. You can insert Markdown cells and code cells. The first you can use for documenting and explaining your results the second you can use writing code snippets that execute the tasks required.  

In this assignment you will implement a Naïve Bayes classifier in Python that will classify emails into spam and non-spam (“ham”) classes.  Your program should be able to train on a given set of spam and “ham” datasets. 
You will work with the datasets available at https://spamassassin.apache.org/old/publiccorpus/. There are three types of files in this location: 
-	easy-ham: non-spam messages typically quite easy to differentiate from spam messages. 
-	hard-ham: non-spam messages more difficult to differentiate 
-	spam: spam messages 

**Execute the cell below to download and extract the data into the environment of the notebook -- it will take a few seconds.** If you chose to use Jupyter notebooks you will have to run the commands in the cell below on your local computer, with Windows you can use 7zip (https://www.7-zip.org/download.html) to decompress the data.



In [1]:
#Download and extract data
#!wget https://spamassassin.apache.org/old/publiccorpus/20021010_easy_ham.tar.bz2
#!wget https://spamassassin.apache.org/old/publiccorpus/20021010_hard_ham.tar.bz2
#!wget https://spamassassin.apache.org/old/publiccorpus/20021010_spam.tar.bz2
#!tar -xjf 20021010_easy_ham.tar.bz2
#!tar -xjf 20021010_hard_ham.tar.bz2
#!tar -xjf 20021010_spam.tar.bz2

*The* data is now in the three folders `easy_ham`, `hard_ham`, and `spam`.

In [2]:
!ls -lah

total 4.5M
drwxrwxr-x 5 root root    9 Feb 17 17:33 .
drwxr-xr-x 8 root root    8 Feb 10 11:15 ..
-rw-r--r-- 1 root root 1.6M Jun 29  2004 20021010_easy_ham.tar.bz2
-rw-r--r-- 1 root root 998K Dec 16  2004 20021010_hard_ham.tar.bz2
-rw-r--r-- 1 root root 1.2M Jun 29  2004 20021010_spam.tar.bz2
-rw-rw-r-- 1 root root  35K Feb 17 17:33 Assignment4.ipynb
drwx--x--x 2  500  500 2.5K Oct 10  2002 easy_ham
drwx--x--x 2 1000 1000  252 Dec 16  2004 hard_ham
drwxr-xr-x 2  500  500  503 Oct 10  2002 spam


### 1. Preprocessing: 
1.	Note that the email files contain a lot of extra information, besides the actual message. Ignore that and run on the entire text. 
2.	We don’t want to train and test on the same data. Split the spam and the ham datasets in a training set and a test set. (`hamtrain`, `spamtrain`, `hamtest`, and `spamtest`) **0.5p**


In [3]:
import os
from sklearn.model_selection import train_test_split

def load_files(dir):
    files = []
    with os.scandir(dir) as it:
        for entry in it:
            with open(entry, 'r',  encoding='iso-8859-1') as f:
                data = f.read()
                files.append(data)
    return files

easy_ham = load_files('easy_ham')
hard_ham = load_files('hard_ham')
spam = load_files('spam')


ham_label = ['ham']*(len(easy_ham + hard_ham))
spam_label = ['spam']*len(spam)

ham_train, ham_test, ham_label_train, ham_label_test = train_test_split(easy_ham + hard_ham, ham_label, test_size=0.3, random_state=42)
spam_train, spam_test, spam_label_train, spam_label_test = train_test_split(spam, spam_label, test_size=0.3, random_state=42)


hard_ham_label = ['hardham']*(len(hard_ham))
easy_ham_label = ['easyham']*len(easy_ham)

hard_ham_train, hard_ham_test, hard_ham_label_train, hard_ham_label_test = train_test_split(hard_ham, hard_ham_label, test_size=0.3, random_state=42)
easy_ham_train, easy_ham_test, easy_ham_label_train, easy_ham_label_test = train_test_split(easy_ham, easy_ham_label, test_size=0.3, random_state=42)

### 2. Write a Python program that: 
1.	Uses four datasets (`hamtrain`, `spamtrain`, `hamtest`, and `spamtest`) 
2.	Trains a Naïve Bayes classifier (e.g. Sklearn) on `hamtrain` and `spamtrain`, that classifies the test sets and reports True Positive and True Negative rates on the `hamtest` and `spamtest` datasets. You can use `CountVectorizer` to transform the email texts into vectors. Please note that there are different types of Naïve Bayes Classifier in SKlearn ([Documentation here](https://scikit-learn.org/stable/modules/naive_bayes.html)). Test two of these classifiers that are well suited for this problem
    - Multinomial Naive Bayes  
    - Bernoulli Naive Bayes. 






In [4]:
#Imports
from sklearn import datasets
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from pprint import pprint



def run_predictions(hamtrain, spamtrain, hamtest, spamtest,
                    hamtrainlabels, spamtrainlabels, hamtestlabels, spamtestlabels,
                    mnb=MultinomialNB(), 
                    bnb=BernoulliNB(), 
                    vectorizer=CountVectorizer(),
                    return_accuracy=False):

    #Train the model using the training sets
    train = vectorizer.fit_transform(hamtrain + spamtrain)

    test_ham = vectorizer.transform(hamtest)
    test_spam = vectorizer.transform(spamtest)
    test_set = vectorizer.transform(hamtest + spamtest)

    #Predict the response for test dataset
    mnb.fit(train, hamtrainlabels + spamtrainlabels)
    bnb.fit(train, hamtrainlabels + spamtrainlabels)

    pred_mnb_ham = mnb.predict(test_ham)
    pred_bnb_ham = bnb.predict(test_ham)

    pred_mnb_spam = mnb.predict(test_spam)
    pred_bnb_spam = bnb.predict(test_spam)

    if return_accuracy:
        return (mnb.score(test_set, hamtestlabels + spamtestlabels), bnb.score(test_set, hamtestlabels + spamtestlabels))
    # Show
    unique, counts = np.unique(pred_mnb_ham, return_counts=True)
    print(f"True positives (Multinomial): {dict(zip(unique, counts))[hamtestlabels[0]]}")
    unique, counts = np.unique(pred_bnb_ham, return_counts=True)
    print(f"True positives (Bernoulli):   {dict(zip(unique, counts))[hamtestlabels[0]]}")

    print()

    unique, counts = np.unique(pred_mnb_spam, return_counts=True)
    print(f"True negatives (Multinomial): {dict(zip(unique, counts))[spamtestlabels[0]]}")
    unique, counts = np.unique(pred_bnb_spam, return_counts=True)
    print(f"True negatives (Bernoulli):   {dict(zip(unique, counts))[spamtestlabels[0]]}")

    print()

    print(f"Accuracy for multinomial test: {mnb.score(test_set, hamtestlabels + spamtestlabels):,.2f}")
    print(f"Accuracy for bernoulli test: {bnb.score(test_set, hamtestlabels + spamtestlabels):,.2f}")

    print()

In [5]:
run_predictions(ham_train, spam_train, ham_test, spam_test, ham_label_train, spam_label_train, ham_label_test, spam_label_test)

True positives (Multinomial): 840
True positives (Bernoulli):   839

True negatives (Multinomial): 131
True negatives (Bernoulli):   30

Accuracy for multinomial test: 0.98
Accuracy for bernoulli test: 0.88



a) Explain how the classifiers differ. What different interpretations do they have? **1p** 

The classifiers used above are Multinomial and Bernoulli Naive Bayes, the main difference between the two is that the multinomial classifier is based on the occurence of discrete features, e.g. something that can undertake any value - let's say the number of spam-related words in an email. The more frequent that feature is in the email, the more likely it is to be considered spam. The Bernoulli classifier, however, only takes into account if a feature is present or not - in our case a spam-related word/formulation - not how frequently this feature occurs.

As shown in the prints above the multinomial classifier is more accurate, this can thus be explained by the fact that it doesn't only consider whether a feature is present or not, but how frequent it is. 

For (a very simplified) example, an email that is spam might be a money-related scam, so the email might include many dollar signs. The multinomial classifier realises that it is a lot more frequent in spam emails than ham emails - but the dollar sign can of course still be present in a ham email without being in a malicious context. The Bernoulli classifier cannot consider this, as it will only check whether the dollar sign is present or not in the email.

### 3.Run your program on 
-	Spam versus easy-ham 
-	Spam versus (hard-ham + easy-ham). 
-   Discuss your results **2.5p** 

** Spam versus easy-ham **

In [6]:
run_predictions(easy_ham_train, spam_train, easy_ham_test, spam_test, easy_ham_label_train, spam_label_train, easy_ham_label_test, spam_label_test)

True positives (Multinomial): 766
True positives (Bernoulli):   764

True negatives (Multinomial): 129
True negatives (Bernoulli):   69

Accuracy for multinomial test: 0.98
Accuracy for bernoulli test: 0.91



** Spam versus (hard-ham + easy-ham). **

In [7]:
#same set as question 2
run_predictions(ham_train, spam_train, ham_test, spam_test, ham_label_train, spam_label_train, ham_label_test, spam_label_test)

True positives (Multinomial): 840
True positives (Bernoulli):   839

True negatives (Multinomial): 131
True negatives (Bernoulli):   30

Accuracy for multinomial test: 0.98
Accuracy for bernoulli test: 0.88



There is not much of a difference between the two looking on the accuracy. However the spam prediction with the Bernoulli classifier finds more than twice as many spam-emails in the easy-ham set compared to the combined set. As previously stated, this is likely because it doesn't take into account the **frequency** of the feature, only whether the spam-related word is present or not. The hard-ham emails might have more spam-like features without actually being spam, thus it becomes more necessary to also consider the frequency of the spam-related words.

# 4.	To avoid classification based on common and uninformative words it is common to filter these out. 

**a.** Argue why this may be useful. Try finding the words that are too common/uncommon in the dataset. **1p** 

In [8]:
from collections import Counter 
from collections import OrderedDict


ham = easy_ham + hard_ham



def get_most_uncommon_common(dataset, low, high):
    all_common = dict()
    for email in dataset:
        words = email.split()
        counter = Counter(words)
        most_occur = counter.most_common(10)
        for (w, i) in most_occur:
            if not w.isalpha():
                continue
            if w in all_common:
                all_common[w] += i
            else:
                all_common[w] = i

    d_descending = OrderedDict(sorted(all_common.items(), 
                                    key=lambda kv: kv[1], reverse=True))
    cleaned = {key:val for key, val in dict(d_descending).items() if val < low or val > high}
    return cleaned

print('Ham most uncommon and common: ' , get_most_uncommon_common(ham, 10, 2000))
print('Spam most uncommon and common: ' ,get_most_uncommon_common(spam, 10, 1000))

Ham most uncommon and common:  {'the': 25655, 'to': 14412, 'from': 14309, 'with': 14193, 'for': 13702, 'by': 13575, 'of': 11293, 'and': 10213, 'a': 9710, 'Sep': 9249, 'Oct': 5312, 'in': 5090, 'Aug': 4959, 'id': 4024, 'is': 2570, 'that': 2446, 'I': 2244, 'SA': 9, 'bees': 9, 'Load': 9, 'player': 9, 'play': 9, 'military': 9, 'memory': 8, 'other': 8, 'training': 8, 'features': 8, 'XDegrees': 8, 'something': 8, 'write': 8, 'speech': 8, 'them': 8, 'files': 8, 'And': 8, 'themes': 8, 'WordInfo': 8, 'web': 8, 'spamd': 8, 'bytecode': 8, 'into': 7, 'yield': 7, 'percentages': 7, 'unique': 7, 'dot': 7, 'gif': 7, 'floor': 7, 'SubSection': 7, 'messages': 7, 'number': 7, 'Mom': 7, 'Judo': 7, 'clues': 7, 'Tried': 7, 'dequeue': 7, 'fetching': 7, 'object': 7, 'Solaris': 7, 'inbound': 7, 'CatchUp': 7, 'but': 6, 'MARLA': 6, 'same': 6, 'cab': 6, 'timtest': 6, 'File': 6, 'top': 6, 'once': 6, 'trained': 6, 'procmail': 6, 'Right': 6, 'show': 6, 'shows': 6, 'lizard': 6, 'store': 6, 'honor': 6, 'Bad': 6, 'razor'

In the lists above you can see the most common and uncommon words for both ham and spam. The boundaries used are if a word appears more than 2000 times in the ham set or 1000 times in the spam set its very common and less then 10 times its very uncommon. We took a lower upperboundry for the spam set as the spam set contains fewer emails.
***

**b.** Use the parameters in Sklearn’s `CountVectorizer` to filter out these words. Update the program from point 3 and run it on your data and report and discuss your results. You have two options to do this in Sklearn: either using the words found in part (a) or letting Sklearn do it for you. Argue for your decision-making. **1p** 

In [9]:
#stopwords = list(set(list(get_noise(ham_train, 100, 2000).keys()) + list(get_noise(spam_train, 100, 500).keys())))
#stopwords = [x.lower() for x in stopwords]
#run_predictions(easy_ham_train, spam_train, easy_ham_test, spam_test, easy_ham_label_train, spam_label_train, easy_ham_label_test, spam_label_test)


low = [0.01, 0.02, 0.04, 0.08]
high = [0.9, 0.92, 0.94, 0.98]

best_accuracy = ()
best_values = tuple()

for i in low:
    for k in high:
        result = run_predictions(ham_train, spam_train, ham_test, spam_test, ham_label_train, spam_label_train, ham_label_test, spam_label_test, vectorizer=CountVectorizer(min_df=i, max_df = k), return_accuracy=True)
        print('low:', i, 'high:', k, 'multinomial:', round(result[0], 2), 'binomial:', round(result[1], 2))

low: 0.01 high: 0.9 multinomial: 0.98 binomial: 0.98
low: 0.01 high: 0.92 multinomial: 0.98 binomial: 0.98
low: 0.01 high: 0.94 multinomial: 0.98 binomial: 0.98
low: 0.01 high: 0.98 multinomial: 0.98 binomial: 0.98
low: 0.02 high: 0.9 multinomial: 0.97 binomial: 0.97
low: 0.02 high: 0.92 multinomial: 0.97 binomial: 0.97
low: 0.02 high: 0.94 multinomial: 0.97 binomial: 0.97
low: 0.02 high: 0.98 multinomial: 0.97 binomial: 0.97
low: 0.04 high: 0.9 multinomial: 0.95 binomial: 0.93
low: 0.04 high: 0.92 multinomial: 0.95 binomial: 0.93
low: 0.04 high: 0.94 multinomial: 0.95 binomial: 0.93
low: 0.04 high: 0.98 multinomial: 0.95 binomial: 0.94
low: 0.08 high: 0.9 multinomial: 0.94 binomial: 0.9
low: 0.08 high: 0.92 multinomial: 0.94 binomial: 0.9
low: 0.08 high: 0.94 multinomial: 0.95 binomial: 0.91
low: 0.08 high: 0.98 multinomial: 0.94 binomial: 0.91


We decided to use SkLearns own defined parameters when filtering out words to minimize the human factor in the result. To get a good match for the values
we tried a few different `min_df` and `max_df` values. Doing this we found that using "harsher" values (e.g. 0.1 and 0.9+) produced the best results - see above. 

### 5. Eeking out further performance
**a.**  Use a lemmatizer to normalize the text (for example from the `nltk` library). For one implementation look at the documentation ([here](https://scikit-learn.org/stable/modules/feature_extraction.html#customizing-the-vectorizer-classes)). Run your program again and answer the following questions: 
  - Why can lemmatization help?
  -	Does the result improve from 3 and 4? Discuss. **1.5p** 







In [10]:
import nltk
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer 
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
class LemmaTokenizer:
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

vect1 = CountVectorizer(tokenizer=LemmaTokenizer())
vect2 = CountVectorizer(tokenizer=LemmaTokenizer(), min_df=0.01, max_df=0.98)

run_predictions(ham_train, spam_train, ham_test, spam_test, ham_label_train, spam_label_train, ham_label_test, spam_label_test, vectorizer=vect1)
run_predictions(ham_train, spam_train, ham_test, spam_test, ham_label_train, spam_label_train, ham_label_test, spam_label_test, vectorizer=vect2)

True positives (Multinomial): 839
True positives (Bernoulli):   836

True negatives (Multinomial): 126
True negatives (Bernoulli):   51

Accuracy for multinomial test: 0.97
Accuracy for bernoulli test: 0.89

True positives (Multinomial): 824
True positives (Bernoulli):   813

True negatives (Multinomial): 144
True negatives (Bernoulli):   151

Accuracy for multinomial test: 0.98
Accuracy for bernoulli test: 0.97



**Why can lemmatization help?**

Lemmatization can help by grouping different implementations of words, let's take the word "use" for example, in the multinomial classification 
it might group together words like using, used, usage to be a single "class" of words, thus increasing the frequency as they are treated as the same word.
Furthermore, in the Bernoulli classification it can improve since it checks multiple implementations of the word at the same time, thus increasing the probability
that a word class will be detected in multiple emails instead of the different words being treated by themselves.

**Does the result improve from 3 and 4?**

In our case the result is not heavily improved, the only increase in performance is that the Bernoulli classification from 3 is improved by one percentage point. 
***

**b.** The split of the data set into a training set and a test set can lead to very skewed results. Why is this, and do you have suggestions on remedies? 
 What do you expect would happen if your training set were mostly spam messages while your test set were mostly ham messages?  **1p** 

The split can lead to skewed results if the proportions of spam versus ham messages are different in the training and test set, let's use the example from the question for reference. If the training set is mostly spam messages the model will be biased towards spam and not be able to produce a good interpretation of what a ham message looks like and the "spam threshold" will not be balanced. When the model is later tested, it will, as a consequence of this biased training, predict that many ham messages are spam as well.

To remedy this, we can split the datasets beforehand to make sure that the proportions of spam messages versus ham messages are (almost) the same in both the training set and test set. 


**c.** Re-estimate your classifier using `fit_prior` parameter set to `false`, and answer the following questions:
  - What does this parameter mean?
  - How does this alter the predictions? Discuss why or why not. **0.5p** 

In [12]:
ham_prior = len(ham_train) / len(ham_train + spam_train)
spam_prior = 1 - ham_prior


print('No prior provided')
run_predictions(ham_train, spam_train, ham_test, spam_test, ham_label_train, spam_label_train, ham_label_test, spam_label_test, mnb=MultinomialNB(fit_prior=False), bnb=BernoulliNB(fit_prior=False))
print('-------------------')
print('Prior provided')
run_predictions(ham_train, spam_train, ham_test, spam_test, ham_label_train, spam_label_train, ham_label_test, spam_label_test, mnb=MultinomialNB(fit_prior=True, class_prior=[ham_prior, spam_prior]), bnb=BernoulliNB(fit_prior=True, class_prior=[ham_prior, spam_prior]))

No prior provided
True positives (Multinomial): 840
True positives (Bernoulli):   839

True negatives (Multinomial): 132
True negatives (Bernoulli):   31

Accuracy for multinomial test: 0.98
Accuracy for bernoulli test: 0.88

-------------------
Prior provided
True positives (Multinomial): 840
True positives (Bernoulli):   839

True negatives (Multinomial): 131
True negatives (Bernoulli):   30

Accuracy for multinomial test: 0.98
Accuracy for bernoulli test: 0.88



The fit_prior=False parameter means that the classifiers will use a uniform prior. This does barely alter the predictions, when prior is provided the true negatives get one less correct prediction. We believe that this might be because we use the built-in function for splitting data which uses the `random_state` parameter and
shuffles the data before applying the split. If we did the splitting manually or without shuffling the data it might come in handy as the splitting otherwise could become skewed.
***

**d.** The python model includes smoothing (`alpha` parameter ), explain why this can be important. 
  - What would happen if in the training data set the word 'money' only appears in spam examples? What would the model predict about a message containing the word 'money'? Does the prediction depend on the rest of the message and is that reasonable? Explain your reasoning  **1p** 

If we don't use any smoothing the model would in theory instantly predict that a message containing 'money' is spam - no matter the other contents of the message - since there is no training messages containing 'money' that are ham. If we use the default value of 1, we produce a small "wiggle room" that a ham message can include 'money'. If the smoothing value gets unreasonably high the probability will move towards 50/50, which might not be accurate as we can assume that the word 'money' is more frequent in spam messages. When this happens we lose the value of the training set as the model now thinks that 'money' is as frequent in spam and ham messages. It is therefore a good idea to use a rather low value, such as 1. 

[Source](https://towardsdatascience.com/laplace-smoothing-in-na%C3%AFve-bayes-algorithm-9c237a8bdece) 

### What to report and how to hand in.

- You will need to clearly report all results in the notebook in a clear and appropriate way, either using plots or code output (f.x. "print statements"). 
- The notebook must be reproducible, that means, we must be able to use the `Run all` function from the `Runtime` menu and reproduce all your results. **Please check this before handing in.** 
- Save the notebook and share a link to the notebook (Press share in upper left corner, and use `Get link` option. **Please make sure to allow all with the link to open and edit.**
- Edits made after submission deadline will be ignored, graders will recover the last saved version before deadline from the revisions history.
- **Please make sure all cells are executed and all the output is clearly readable/visible to anybody opening the notebook.**

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4d60117c-244d-43ff-9bed-27d4239d5495' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>